In [ ]:
#twitter API details
import tweepy,json
consumerKey = "#####################"
consumerSecret = "#########################"
accesstoken = "##########################"
accesssecret = "################################"
auth= tweepy.OAuthHandler(consumerKey,consumerSecret)
auth.set_access_token(accesstoken,accesssecret)

In [ ]:
tweet_list=[]
class MyStreamListener(tweepy.StreamListener):
    def __init__(self,api=None):
        super(MyStreamListener,self).__init__()
        self.num_tweets=0
        self.file=open("data6.json","w")
    def on_status(self,status):
        tweet=status._json
        self.file.write(json.dumps(tweet)+ '\n')
        tweet_list.append(status)
        self.num_tweets+=1
        if self.num_tweets<500:
            return True
        else:
            return False
        self.file.close()

In [ ]:
l = MyStreamListener()
stream =tweepy.Stream(auth,l)
#this line filters twiiter streams to capture data by keywords
stream.filter(track=['oil','gas','refinery','fuel','Petrol'])

In [ ]:
import pandas as pd
df= pd.read_json('data6.json',lines=True)
df['label'] ='Oil and Gas'
df.head()


,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,possibly_sensitive,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_tweet,display_text_range,label
0,2021-05-26 18:49:14+00:00,1397625894714368007,1397625894714368000,RT @badmusib: I asked blessing to help me prep...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,None,"{'id': 938669612631298048, 'id_str': '93866961...",NaN,NaN,None,NaN,{'created_at': 'Wed May 26 18:31:32 +0000 2021...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2021-05-26 18:49:14.590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oil and Gas
1,2021-05-26 18:49:14+00:00,1397625895226064898,1397625895226064896,"Oil futures finish with a modest gain, up a 4t...","<a href=""http://publicize.wp.com/"" rel=""nofoll...",False,NaN,NaN,NaN,NaN,None,"{'id': 3316713321, 'id_str': '3316713321', 'na...",NaN,NaN,None,NaN,NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,2021-05-26 18:49:14.712,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oil and Gas
2,2021-05-26 18:49:14+00:00,1397625896085770241,1397625896085770240,RT @Boo_Rad13y: Jet Fuel cant melt steel beams...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,None,"{'id': 1208756689710616576, 'id_str': '1208756...",NaN,NaN,None,NaN,{'created_at': 'Wed May 26 14:34:24 +0000 2021...,False,0,0,0,0,"{'hashtags': [{'text': 'MonsterHunter', 'indic...",False,False,low,en,2021-05-26 18:49:14.917,0.0,"{'media': [{'id': 1397560356017524737, 'id_str...",NaN,NaN,NaN,NaN,NaN,NaN,Oil and Gas
3,2021-05-26 18:49:15+00:00,1397625897172168704,1397625897172168704,RT @GFuelEnergy: 🚨🇺🇸 BOGO + GIVEAWAY 🇺🇸🚨\n\n🤩...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,None,"{'id': 1396926000278118401, 'id_str': '1396926...",NaN,NaN,None,NaN,{'created_at': 'Wed May 26 16:00:56 +0000 2021...,False,0,0,0,0,"{'hashtags': [{'text': 'MemorialDay', 'indices...",False,False,low,en,2021-05-26 18:49:15.176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oil and Gas
4,2021-05-26 18:49:15+00:00,1397625898719924231,1397625898719924224,RT @mindingottawa: MP @MartinBowRiver threaten...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,None,"{'id': 220833174, 'id_str': '220833174', 'name...",NaN,NaN,None,NaN,{'created_at': 'Wed May 26 12:17:19 +0000 2021...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2021-05-26 18:49:15.545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oil and Gas


In [ ]:
df.to_csv('oil.csv')

In [ ]:
df_History = pd.read_csv('History.csv')
df_Medical = pd.read_csv('Medical.csv')
df_Sport = pd.read_csv('Sport.csv')
df_Robotics = pd.read_csv('Robotics.csv')
df_Technology =pd.read_csv('Techology.csv')
df_oil = pd.read_csv('oil.csv')

In [ ]:
df_oil = pd.read_csv('oil.csv')

In [ ]:
df_oil = df_oil[['text','label']]

In [ ]:
df_Medical = df_Medical[['text','label']]
df_Medical.head()

,text,label
0,RT @TartarusRespawn: JUST IN: Google Strikes D...,Medical
1,@Montes19Gerardo Posteé en base a eso de hace ...,Medical
2,RT @drmwarner: We have an octogenarian husband...,Medical
3,RT @kmoney20205: Naughty nurse and naughty tea...,Medical
4,RT @rahimshahUSA: KSA Provide Covid Vaccine at...,Medical


In [ ]:
df_Sport = df_Sport[['text','label']]

In [ ]:
df_data = df_data[['text','label']]

In [ ]:
df_Robotics = df_Robotics[['text','label']]

In [ ]:
df_Technology = df_Technology[['text','label']]

In [ ]:
data = pd.concat([df_Histo1,df_oil])

In [ ]:
data.shape

(2113, 38)

In [ ]:
data.to_csv('Train_data.csv')

In [ ]:
import pandas as pd
data = pd.read_csv('Train_data.csv')


In [ ]:
data.head()

,Unnamed: 0,Unnamed: 0.1,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,display_text_range,extended_tweet,possibly_sensitive,extended_entities,label
0,0,0.0,2021-05-26 18:09:23+00:00,1.397616e+18,1.397616e+18,RT @ColumbiaRecords: “BUTTER” by BTS has offic...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,NaN,"{'id': 771919061244604416, 'id_str': '77191906...",NaN,NaN,NaN,NaN,{'created_at': 'Sun May 23 05:26:20 +0000 2021...,False,0.0,0.0,0.0,0.0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2021-05-26 18:09:23.745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,History
1,1,1.0,2021-05-26 18:09:24+00:00,1.397616e+18,1.397616e+18,RT @chartdata: .@ArianaGrande has now surpasse...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,"{'id': 1264263560298762240, 'id_str': '1264263...",NaN,NaN,NaN,NaN,{'created_at': 'Wed May 26 18:08:02 +0000 2021...,False,0.0,0.0,0.0,0.0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2021-05-26 18:09:24.321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,History
2,2,2.0,2021-05-26 18:09:24+00:00,1.397616e+18,1.397616e+18,@FOX10Phoenix @azfamily @abc15,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,18999261.0,18999261.0,FOX10Phoenix,"{'id': 36101474, 'id_str': '36101474', 'name':...",NaN,NaN,NaN,NaN,NaN,True,0.0,0.0,0.0,0.0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,und,2021-05-26 18:09:24.317,1.395444e+18,1.395444e+18,{'created_at': 'Thu May 20 18:18:59 +0000 2021...,"{'url': 'https://t.co/M6JWMOAtYy', 'expanded':...",NaN,NaN,NaN,NaN,History
3,3,3.0,2021-05-26 18:09:24+00:00,1.397616e+18,1.397616e+18,RT @marcorubio: Knew host for SARS in 4 months...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,NaN,"{'id': 1027190315742945281, 'id_str': '1027190...",NaN,NaN,NaN,NaN,{'created_at': 'Wed May 26 17:36:30 +0000 2021...,False,0.0,0.0,0.0,0.0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2021-05-26 18:09:24.349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,History
4,4,4.0,2021-05-26 18:09:24+00:00,1.397616e+18,1.397616e+18,Diesen wirklich bemerkenswerten Roman von @pas...,"<a href=""http://twitter.com/download/iphone"" r...",True,NaN,NaN,NaN,NaN,NaN,"{'id': 37639653, 'id_str': '37639653', 'name':...",NaN,NaN,"{'id': 'b9b77321807927e1', 'url': 'https://api...",NaN,NaN,False,0.0,0.0,0.0,0.0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,de,2021-05-26 18:09:24.315,NaN,NaN,NaN,NaN,"[0, 140]",{'full_text': 'Diesen wirklich bemerkenswerten...,0.0,NaN,History


In [ ]:
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
data = data[['id','text','label']]
data.head()
data.shape

(2113, 3)

In [ ]:
!pip install contractions
import contractions
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import string
%matplotlib inline
!pip install emoji
import emoji
import re


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('words')
words = set(nltk.corpus.words.words())
#words = set(nltk.corpus.words.words())

     |████████████████████████████████| 286kB 6.7MB/s 
     |████████████████████████████████| 327kB 25.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85392 sha256=25e3c773e9c19e0e604c500465ede928da321078c2c5a12569a1649b1227d2d0
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
     |████████████████████████████████| 133kB 5.0MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package wo

In [ ]:
#cleaning and preprocessing of the data
def remove_contraction(tweet):
    contraction_mapping = {"ain't": "is not", "aren't": "are not", "can't": "cannot", "'cause": "because",
                           "could've": "could have", "couldn't": "could not",
                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                           "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will",
                           "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
                           "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
                           "I'll've": "I will have", "I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have", "i'm": "i am",
                           "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
                           "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
                           "ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not",
                           "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
                           "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                           "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
                           "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will",
                           "she'll've": "she will have", "she's": "she is", "should've": "should have",
                           "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                           "so's": "so as",
                           "this's": "this is", "that'd": "that would", "that'd've": "that would have",
                           "that's": "that is",
                           "there'd": "there would", "there'd've": "there would have", "there's": "there is",
                           "here's": "here is",
                           "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
                           "they'll've": "they will have",
                           "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                           "we'd": "we would", "we'd've": "we would have", "we'll": "we will",
                           "we'll've": "we will have",
                           "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
                           "what'll've": "what will have",
                           "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
                           "when've": "when have",
                           "where'd": "where did", "where's": "where is", "where've": "where have",
                           "who'll": "who will",
                           "who'll've": "who will have",
                           "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have",
                           "will've": "will have",
                           "won't": "will not", "won't've": "will not have", "would've": "would have",
                           "wouldn't": "would not",
                           "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                           "y'all'd've": "you all would have",
                           "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would",
                           "you'd've": "you would have", "you'll": "you will",
                           "you'll've": "you will have", "you're": "you are", "you've": "you have"}
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        tweet = tweet.replace(s, "'")
    tweet = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in tweet.split(" ")])
    return tweet

def clean_repeat_patterns_lower(tweet,remove_repeat_text=True, remove_patterns_text=True, is_lower=True):

    RE_PATTERNS = {
    ' american ':
        [
            'amerikan'
        ],

    ' adolf ':
        [
            'adolf'
        ],


    ' hitler ':
        [
            'hitler'
        ],

    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*', 'f**k','fu*k',
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'
        ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$',
            '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s','a55', '@$$'
        ],

    ' ass hole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole', 'a**hole'
        ],

    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h', 'b!tch', 'bi+ch', 'l3itch'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],

    ' trans gender':
        [
            'transgender'
        ],

    ' gay ':
        [
            'gay'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k', 'dik'
        ],

    ' suck ':
        [
            'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bull shit ':
        [
            'bullsh\*t', 'bull\$hit'
        ],

    ' homo sex ual':
        [
            'homosexual'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots'
                                                                                      'i d i o t'
        ],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t', '$h1t'
        ],

    ' shit hole ':
        [
            'shythole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            ' raped'
        ],

    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],

    ' ass head':
        [
            'butthead'
        ],

    ' sex ':
        [
            'sexy', 's3x', 'sexuality'
        ],


    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],

    ' shut the fuck up':
        [
            'stfu', 'st*u'
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses', 'p*ssy'
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' mother fucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker',
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],
    ' fucking ':
        [
            'f*$%-ing'
        ],
}

    if is_lower:
      tweet=tweet.lower()

    if remove_patterns_text:
      for target, patterns in RE_PATTERNS.items():
        for pat in patterns:
          tweet=str(tweet).replace(pat, target)

    if remove_repeat_text:
     tweet = re.sub(r'(.)\1{2,}', r'\1', tweet)

     tweet = str(tweet).replace("\n", " ")
     tweet = re.sub(r'[^\w\s]',' ',tweet)
     tweet = re.sub('[0-9]',"",tweet)
     tweet = re.sub(" +", " ", tweet)
     tweet = re.sub("([^\x00-\x7F])+"," ",tweet)
     tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
     tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
     shortword= re.compile(r'\W*\b\w{1,3}\b')
     tweet = shortword.sub('', tweet)
     tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
     return tweet



def remove_emojis(tweet):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U0001F1F2-\U0001F1F4"
        u"\U0001F1E6-\U0001F1FF"
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', tweet)

def prepare_stopwords_list():
    stopword_list=STOP_WORDS
    potential_stopwords=['editor', 'reference', 'thank', 'work','find', 'good', 'know', 'like', 'look', 'thing', 'want', 'time', 'list', 'section','wikipedia', 'doe', 'add','new', 'try', 'think', 'write','use', 'user', 'way', 'page']
    for word in potential_stopwords:
        stopword_list.add(word)
    return(stopword_list)

def remove_stop_words(tweet, remove_stop=True):
    stop_words=prepare_stopwords_list()
    output = ""
    if remove_stop:
      tweet=tweet.split(" ")
      for word in tweet:
        if word not in stop_words:
          output=output + " " + word
    else :
      output=text
    return str(output.strip())

def lemmatize(tweet, lemmatization=True):
  wordnet_lemmatizer = WordNetLemmatizer()
  output=""
  if lemmatization:
    tweet=tweet.split(" ")
    for word in tweet:
       word1 = wordnet_lemmatizer.lemmatize(word, pos = "n")
       word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
       word3 = wordnet_lemmatizer.lemmatize(word2, pos = "a")
       word4 = wordnet_lemmatizer.lemmatize(word3, pos = "r")
       output=output + " " + word4
  else:
    output=tweet
  return str(output.strip())

def clean_text_column(text_column):
    return text_column.apply(lambda x: clean_text(x))

def clean_text(tweet):
    tweet=remove_contraction(tweet)
    tweet=clean_repeat_patterns_lower(tweet)
    tweet=remove_emojis(tweet)
    tweet=remove_stop_words(tweet)
    tweet=lemmatize(tweet)
    return tweet





In [ ]:
data['tweet'] = data['text'].map(lambda x: clean_text(x))

In [ ]:
data.head()

,id,text,label,tweet
0,NaN,Unless it means abolishing the filibster.,Medical,
1,1.397615e+18,@launchzoneann Just keep believing and keep wo...,Technology,believe work love sooner late
2,NaN,Los héroes si exiten y no son esos de camuflaj...,Oil and Gas,
3,NaN,@tvg_infantil_xa As grandes tecnolóxicas tende...,Oil and Gas,tvg_infantil_xa
4,NaN,Nova Scotia: Ya'll are fucking insane.,Medical,nova scotia insane


In [ ]:
data['tokenized'] = data['tweet'].apply(word_tokenize)
data.head()

,id,text,label,tweet,tokenized
0,NaN,Unless it means abolishing the filibster.,Medical,,[]
1,1.397615e+18,@launchzoneann Just keep believing and keep wo...,Technology,believe work love sooner late,"[believe, work, love, sooner, late]"
2,NaN,Los héroes si exiten y no son esos de camuflaj...,Oil and Gas,,[]
3,NaN,@tvg_infantil_xa As grandes tecnolóxicas tende...,Oil and Gas,tvg_infantil_xa,[tvg_infantil_xa]
4,NaN,Nova Scotia: Ya'll are fucking insane.,Medical,nova scotia insane,"[nova, scotia, insane]"


In [ ]:
data['pos_tags'] = data['tokenized'].apply(nltk.tag.pos_tag)
data.head()

,id,text,label,tweet,tokenized,pos_tags
0,NaN,Unless it means abolishing the filibster.,Medical,,[],[]
1,1.397615e+18,@launchzoneann Just keep believing and keep wo...,Technology,believe work love sooner late,"[believe, work, love, sooner, late]","[(believe, JJ), (work, NN), (love, VBD), (soon..."
2,NaN,Los héroes si exiten y no son esos de camuflaj...,Oil and Gas,,[],[]
3,NaN,@tvg_infantil_xa As grandes tecnolóxicas tende...,Oil and Gas,tvg_infantil_xa,[tvg_infantil_xa],"[(tvg_infantil_xa, NN)]"
4,NaN,Nova Scotia: Ya'll are fucking insane.,Medical,nova scotia insane,"[nova, scotia, insane]","[(nova, JJ), (scotia, NN), (insane, NN)]"


In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
data['wordnet_pos'] = data['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
data.head()

,id,text,label,tweet,tokenized,pos_tags,wordnet_pos
0,NaN,Unless it means abolishing the filibster.,Medical,,[],[],[]
1,1.397615e+18,@launchzoneann Just keep believing and keep wo...,Technology,believe work love sooner late,"[believe, work, love, sooner, late]","[(believe, JJ), (work, NN), (love, VBD), (soon...","[(believe, a), (work, n), (love, v), (sooner, ..."
2,NaN,Los héroes si exiten y no son esos de camuflaj...,Oil and Gas,,[],[],[]
3,NaN,@tvg_infantil_xa As grandes tecnolóxicas tende...,Oil and Gas,tvg_infantil_xa,[tvg_infantil_xa],"[(tvg_infantil_xa, NN)]","[(tvg_infantil_xa, n)]"
4,NaN,Nova Scotia: Ya'll are fucking insane.,Medical,nova scotia insane,"[nova, scotia, insane]","[(nova, JJ), (scotia, NN), (insane, NN)]","[(nova, a), (scotia, n), (insane, n)]"


In [ ]:
wnl = WordNetLemmatizer()
data['lemmatized'] = data['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
data.head()

,id,text,label,tweet,tokenized,pos_tags,wordnet_pos,lemmatized
0,NaN,Unless it means abolishing the filibster.,Medical,,[],[],[],[]
1,1.397615e+18,@launchzoneann Just keep believing and keep wo...,Technology,believe work love sooner late,"[believe, work, love, sooner, late]","[(believe, JJ), (work, NN), (love, VBD), (soon...","[(believe, a), (work, n), (love, v), (sooner, ...","[believe, work, love, sooner, late]"
2,NaN,Los héroes si exiten y no son esos de camuflaj...,Oil and Gas,,[],[],[],[]
3,NaN,@tvg_infantil_xa As grandes tecnolóxicas tende...,Oil and Gas,tvg_infantil_xa,[tvg_infantil_xa],"[(tvg_infantil_xa, NN)]","[(tvg_infantil_xa, n)]",[tvg_infantil_xa]
4,NaN,Nova Scotia: Ya'll are fucking insane.,Medical,nova scotia insane,"[nova, scotia, insane]","[(nova, JJ), (scotia, NN), (insane, NN)]","[(nova, a), (scotia, n), (insane, n)]","[nova, scotia, insane]"


In [ ]:
data['lemma_str'] = [' '.join(map(str,l)) for l in data['lemmatized']]
data.head()

,id,text,label,tweet,tokenized,pos_tags,wordnet_pos,lemmatized,lemma_str
0,NaN,Unless it means abolishing the filibster.,Medical,,[],[],[],[],
1,1.397615e+18,@launchzoneann Just keep believing and keep wo...,Technology,believe work love sooner late,"[believe, work, love, sooner, late]","[(believe, JJ), (work, NN), (love, VBD), (soon...","[(believe, a), (work, n), (love, v), (sooner, ...","[believe, work, love, sooner, late]",believe work love sooner late
2,NaN,Los héroes si exiten y no son esos de camuflaj...,Oil and Gas,,[],[],[],[],
3,NaN,@tvg_infantil_xa As grandes tecnolóxicas tende...,Oil and Gas,tvg_infantil_xa,[tvg_infantil_xa],"[(tvg_infantil_xa, NN)]","[(tvg_infantil_xa, n)]",[tvg_infantil_xa],tvg_infantil_xa
4,NaN,Nova Scotia: Ya'll are fucking insane.,Medical,nova scotia insane,"[nova, scotia, insane]","[(nova, JJ), (scotia, NN), (insane, NN)]","[(nova, a), (scotia, n), (insane, n)]","[nova, scotia, insane]",nova scotia insane


In [ ]:
data['label'].value_counts()

Oil and Gas    500
Medical        500
Sport          498
History        416
Robotics       175
Technology      24
Name: label, dtype: int64

In [ ]:
cleanup_nums = {"label":{"Robotics": 0, "Medical": 1,'Oil and Gas':2,'Sport':3,'History':4,'Technology':5}}


#"Robotics": 0, "Medical": 1,'Oil and Gas':2,'Sport':3,'History':4,'Technology':5

In [ ]:
data = data.replace(cleanup_nums)
data.head()

,id,text,label,tweet,tokenized,pos_tags,wordnet_pos,lemmatized,lemma_str
0,NaN,Unless it means abolishing the filibster.,1,,[],[],[],[],
1,1.397615e+18,@launchzoneann Just keep believing and keep wo...,5,believe work love sooner late,"[believe, work, love, sooner, late]","[(believe, JJ), (work, NN), (love, VBD), (soon...","[(believe, a), (work, n), (love, v), (sooner, ...","[believe, work, love, sooner, late]",believe work love sooner late
2,NaN,Los héroes si exiten y no son esos de camuflaj...,2,,[],[],[],[],
3,NaN,@tvg_infantil_xa As grandes tecnolóxicas tende...,2,tvg_infantil_xa,[tvg_infantil_xa],"[(tvg_infantil_xa, NN)]","[(tvg_infantil_xa, n)]",[tvg_infantil_xa],tvg_infantil_xa
4,NaN,Nova Scotia: Ya'll are fucking insane.,1,nova scotia insane,"[nova, scotia, insane]","[(nova, JJ), (scotia, NN), (insane, NN)]","[(nova, a), (scotia, n), (insane, n)]","[nova, scotia, insane]",nova scotia insane


In [ ]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords, wordnet
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [ ]:

Train_X = None
Train_Y = None
Test_X = None
Test_Y = None
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(data['lemma_str'],data['label'],test_size=0.2)

In [ ]:
Encoder=None
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect=None
count_vect = CountVectorizer(lowercase=False)
X_train_counts = count_vect.fit_transform(data['lemma_str'].tolist())
X_train_counts.shape

(2113, 2853)

In [ ]:
Tfidf_vect=None
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data['lemma_str'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
print(Tfidf_vect.vocabulary_)
print(Train_X_Tfidf.shape)

{'believe': 256, 'work': 2827, 'love': 1535, 'sooner': 2377, 'late': 1462, 'tvg_infantil_xa': 2673, 'nova': 1759, 'scotia': 2249, 'insane': 1331, 'wish': 2821, 'endless': 862, 'happiness': 1184, 'health': 1201, 'know': 1443, 'random': 2065, 'exclusive': 921, 'tell': 2542, 'grapefruit': 1150, 'lavender': 1466, 'essential': 896, 'create': 614, 'wonderful': 2825, 'scent': 2237, 'succulent': 2478, 'citrus': 471, 'body': 300, 'monique_c': 1676, 'igor_larionovii': 1272, 'excellent': 917, 'people': 1858, 'right': 2172, 'forget': 1049, 'crash': 612, 'price': 1972, 'nearly': 1733, 'marriage': 1589, 'candidate': 385, 'doctor': 779, 'ready': 2077, 'heart': 1204, 'care': 398, 'slugger': 2353, 'hybrid': 1258, 'scandium': 2232, 'softball': 2366, 'sport': 2401, 'second': 2259, 'chance': 434, 'league': 1473, 'false': 967, 'start': 2422, 'arm': 154, 'superior': 2489, 'ship': 2303, 'navigation': 1729, 'sail': 2211, 'world': 2831, 'fake': 963, 'news': 1742, 'data': 655, 'validation': 2734, 'machine': 154

In [ ]:
#n-gram Tokenization
import pickle
tfidf_vect_ngram = None
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3), max_features=8000)
tfidf_vect_ngram.fit(data['lemma_str'])
Train_X_ngram =  tfidf_vect_ngram.transform(Train_X)
Test_X_ngram =  tfidf_vect_ngram.transform(Test_X)

with open('vectorizer1.pkl', 'wb') as fin:
    pickle.dump(tfidf_vect_ngram, fin)

In [ ]:
print(Train_X_ngram.shape)
print(Train_X_ngram)

(1690, 8000)
  (0, 6854)	0.31754291759188236
  (0, 6853)	0.31754291759188236
  (0, 6849)	0.25756580719971495
  (0, 5680)	0.31754291759188236
  (0, 5679)	0.2622625642595767
  (0, 2473)	0.22377866471860922
  (0, 2318)	0.23220489920954088
  (0, 2046)	0.6350858351837647
  (0, 316)	0.2346223875934239
  (1, 5807)	0.7385649015217742
  (1, 5721)	0.2871138990424329
  (1, 2200)	0.6099897500915761
  (2, 6348)	0.34078189906607054
  (2, 6347)	0.34078189906607054
  (2, 4694)	0.34078189906607054
  (2, 4693)	0.34078189906607054
  (2, 4691)	0.2871703508999439
  (2, 1751)	0.32343016316735335
  (2, 1478)	0.34078189906607054
  (2, 1477)	0.34078189906607054
  (2, 1476)	0.34078189906607054
  (3, 7612)	0.2782836569372748
  (3, 7611)	0.2782836569372748
  (3, 7606)	0.2540607597636237
  (3, 7141)	0.2782836569372748
  :	:
  (1688, 5639)	0.33983486410701597
  (1688, 5638)	0.33983486410701597
  (1688, 5636)	0.3225313487873301
  (1688, 4840)	0.3225313487873301
  (1688, 2781)	0.33983486410701597
  (1688, 2774)	0.267

In [ ]:
X_train = Train_X_ngram
Y_train = Train_Y
X_test = Test_X_ngram
Y_test = Test_Y

In [ ]:
models = [
    svm.SVC(C=1.0, kernel='linear', degree=10, gamma='auto'),
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

In [ ]:
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
y_preds=[]
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, X_train, Train_Y, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy*100))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [ ]:
cv_df.groupby('model_name').accuracy.mean()

model_name
LinearSVC                 73.372781
LogisticRegression        71.420118
MultinomialNB             69.881657
RandomForestClassifier    57.041420
SVC                       72.248521
Name: accuracy, dtype: float64

In [ ]:
print(cv_df)

                model_name  fold_idx   accuracy
0                      SVC         0  70.414201
1                      SVC         1  71.005917
2                      SVC         2  76.331361
3                      SVC         3  71.301775
4                      SVC         4  72.189349
5   RandomForestClassifier         0  57.692308
6   RandomForestClassifier         1  56.804734
7   RandomForestClassifier         2  56.508876
8   RandomForestClassifier         3  56.213018
9   RandomForestClassifier         4  57.988166
10               LinearSVC         0  70.710059
11               LinearSVC         1  72.189349
12               LinearSVC         2  77.514793
13               LinearSVC         3  73.964497
14               LinearSVC         4  72.485207
15           MultinomialNB         0  70.710059
16           MultinomialNB         1  68.047337
17           MultinomialNB         2  71.597633
18           MultinomialNB         3  70.414201
19           MultinomialNB         4  68

In [ ]:
lc =LinearSVC()

In [ ]:
model =lc.fit(X_train,Train_Y)

In [ ]:
Y_pred =model.predict(Test_X_ngram)

In [ ]:
import pickle
filename = 'classify_model_1.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Test_Y,Y_pred)

0.7565011820330969